# Relatório de Análise de Dados Socioeconômicos da PORDATA

Este notebook apresenta a análise de dados socioeconômicos obtidos da PORDATA (Base de Dados de Portugal Contemporâneo), focando em indicadores relacionados à saúde e condições econômicas.

## Introdução

Este relatório apresenta a análise de dados socioeconômicos obtidos da PORDATA (Base de Dados de Portugal Contemporâneo), focando em indicadores relacionados à saúde e condições econômicas. O projeto foi desenvolvido como parte do trabalho prático da disciplina de Elementos de Inteligência Artificial e Ciência de Dados da Universidade da Beira Interior.

A análise foi estruturada em seis fases principais: recolha de dados, limpeza e pré-processamento, exploração de dados, análise estatística, análise de relações entre variáveis e validação dos resultados. Cada fase empregou técnicas específicas implementadas através de scripts Python, permitindo uma abordagem sistemática e reproduzível para a extração de conhecimento a partir dos dados.

Os datasets analisados incluem:
- Ganho médio mensal
- Esperança de vida
- Despesas com saúde
- Percepção de saúde
- Taxa de mortalidade evitável

In [ ]:
# Importação das bibliotecas necessárias para o projeto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Configurações para visualizações
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set_palette('viridis')

# Definir diretório base
DIRETORIO_BASE = "C:/Users/fuguz/DATASETS"

## 1. Fase de Recolha de Dados

### Técnicas Utilizadas

A fase de recolha de dados foi implementada através do script `recolha_dados.py`, que utiliza técnicas de manipulação de arquivos e leitura de dados estruturados. O script emprega a biblioteca `pandas` para carregar os datasets em formato CSV e a biblioteca `os` para navegação no sistema de arquivos.

Uma técnica importante implementada nesta fase foi a detecção automática de codificação de caracteres, permitindo lidar com diferentes formatos de codificação (UTF-16LE e Latin-1) frequentemente encontrados em datasets com caracteres especiais do português. Esta abordagem foi implementada através de um mecanismo de tratamento de exceções que tenta diferentes codificações quando a leitura inicial falha.

### Justificativa

A implementação de um processo automatizado de recolha de dados foi essencial para garantir a reprodutibilidade da análise e facilitar futuras atualizações com novos dados. A função `recolha_dados()` centraliza este processo, permitindo que outros scripts importem os dados de forma consistente.

A técnica de detecção automática de codificação foi necessária devido à presença de caracteres especiais nos nomes dos datasets e nos valores textuais, como "ESPERANÇADEVIDA" e "PERCEÇAODESAUDE". Sem este tratamento, os caracteres especiais poderiam ser interpretados incorretamente, comprometendo a integridade dos dados.

### Resultados

A função `recolha_dados()` conseguiu carregar com sucesso os cinco datasets da PORDATA, identificando automaticamente a codificação correta para cada arquivo. Os datasets foram armazenados em um dicionário Python, onde as chaves são os caminhos dos arquivos e os valores são os dataframes correspondentes.

Esta fase estabeleceu a base para todas as análises subsequentes, garantindo que os dados brutos estivessem disponíveis em um formato adequado para processamento.

In [ ]:
# Implementação da função recolha_dados()
def recolha_dados():
    arquivos_desejados = [
        'DESPESASAUDE.csv',
        'ESPERANÇADEVIDA.csv',
        'GANHOMEDIOMENSAL.csv',
        'PERCEÇAODESAUDE.csv',
        'TAXADEMORTALIDADEVITAVEL.csv'
    ]

    datasets = {}

    # Usamos a variável global DIRETORIO_BASE
    for raiz, dirs, arquivos in os.walk(DIRETORIO_BASE):
        for nome_ficheiro in arquivos:
            if nome_ficheiro in arquivos_desejados:
                caminho = os.path.join(raiz, nome_ficheiro)

                try:
                    # Tentativa de leitura com codificação 'utf-16le'
                    df = pd.read_csv(caminho, encoding='utf-16le')
                    datasets[caminho] = df
                    print(f"Arquivo carregado de {caminho}")
                except UnicodeDecodeError:
                    try:
                        # Tentativa com codificação 'latin1'
                        df = pd.read_csv(caminho, encoding='latin1')
                        datasets[caminho] = df
                        print(f"Arquivo carregado de {caminho} com codificação 'latin1'")
                    except Exception as e:
                        print(f"Erro ao ler {caminho} com codificação 'latin1': {e}")
                except Exception as e:
                    print(f"Erro ao ler {caminho}: {e}")

    return datasets

# Executar a função para carregar os datasets
datasets = recolha_dados()

# Exibir informações básicas sobre os datasets carregados
for caminho, df in datasets.items():
    nome = os.path.basename(caminho)
    print(f"\nDataset: {nome}")
    print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
    print(f"Colunas: {', '.join(df.columns)}")

## 2. Fase de Limpeza e Pré-processamento

### Técnicas Utilizadas

A limpeza e pré-processamento dos dados foram implementados no script `limpeza_dados.py`, que utiliza técnicas de transformação e filtragem de dados. A função principal `limpar_preprocessar()` aplica uma série de operações para preparar os dados para análise:

1. Padronização de nomes de colunas para facilitar o acesso
2. Conversão de tipos de dados (anos para inteiros, valores para numéricos)
3. Tratamento de valores ausentes (remoção de linhas sem valores e preenchimento de campos categóricos vazios)
4. Adição de metadados específicos para cada dataset (indicador e unidade de medida)

### Justificativa

A fase de limpeza e pré-processamento é crucial para garantir a qualidade e consistência dos dados antes da análise. A padronização dos nomes de colunas foi necessária para unificar a estrutura dos diferentes datasets, facilitando a integração posterior.

A conversão de tipos de dados foi implementada para garantir que operações matemáticas e estatísticas pudessem ser aplicadas corretamente. Por exemplo, a conversão da coluna "Ano" para inteiro permite ordenação cronológica adequada, enquanto a conversão da coluna "Valor" para numérico possibilita cálculos estatísticos.

O tratamento de valores ausentes foi essencial para evitar distorções nas análises estatísticas. A decisão de remover linhas sem valores na coluna principal ("Valor") foi tomada para preservar a integridade das análises, enquanto o preenchimento de campos categóricos vazios com strings vazias foi adotado para manter a consistência estrutural.

### Resultados

O processo de limpeza resultou em datasets estruturalmente consistentes e prontos para análise. Cada dataset foi salvo em formato CSV com o sufixo "_limpo", preservando os dados originais e criando versões processadas para as fases subsequentes.

A função `limpar_preprocessar()` gerou um relatório detalhado do processo, incluindo o número de linhas removidas por valores ausentes e as transformações aplicadas a cada dataset. Este relatório foi salvo no arquivo "resumo_limpeza.txt", permitindo a verificação da qualidade do processo.

In [ ]:
# Implementação da função limpar_preprocessar()
def limpar_preprocessar(df, nome_dataset):
    print(f"\n{'='*50}")
    print(f"Limpeza e pré-processamento do dataset: {nome_dataset}")
    print(f"{'='*50}")

    # Cópia do dataframe original para não modificá-lo
    df_limpo = df.copy()

    # 1. Renomear colunas para facilitar o acesso
    colunas_originais = df_limpo.columns.tolist()
    colunas_novas = [
        'Ano', 'Pais', 'Regiao', 'Filtro1', 'Filtro2', 'Filtro3', 'Escala', 'Simbolo', 'Valor'
    ]

    # Verificar se o número de colunas corresponde
    if len(colunas_originais) == len(colunas_novas):
        df_limpo.columns = colunas_novas
        print(f"Colunas renomeadas: {colunas_originais} -> {colunas_novas}")
    else:
        print(f"Erro ao renomear colunas: número de colunas não corresponde ({len(colunas_originais)} vs {len(colunas_novas)})")

    # 2. Converter a coluna Ano para inteiro (quando possível)
    try:
        # Primeiro, remover valores não numéricos
        df_limpo['Ano'] = pd.to_numeric(df_limpo['Ano'], errors='coerce')
        # Converter para inteiro, mantendo NaN onde necessário
        df_limpo['Ano'] = df_limpo['Ano'].astype('Int64')
        print("Coluna 'Ano' convertida para inteiro")
    except Exception as e:
        print(f"Erro ao converter coluna 'Ano' para inteiro: {e}")

    # 3. Tratar valores ausentes
    df_limpo = df_limpo.dropna(subset=['Valor'])
    print(f"Linhas removidas por valor ausente: {len(df) - len(df_limpo)}")

    for col in ['Pais', 'Regiao', 'Filtro1', 'Filtro2', 'Filtro3', 'Escala', 'Simbolo']:
        df_limpo[col] = df_limpo[col].fillna('')

    # 4. Converter a coluna Valor para numérico
    df_limpo['Valor'] = pd.to_numeric(df_limpo['Valor'], errors='coerce')
    print("Coluna 'Valor' convertida para numérico")

    # 5. Adicionar metadados específicos para cada dataset
    if nome_dataset == 'GANHOMEDIOMENSAL':
        df_limpo['Indicador'] = 'Ganho Médio Mensal'
        df_limpo['Unidade'] = 'euros'
    elif nome_dataset == 'ESPERANÇADEVIDA':
        df_limpo['Indicador'] = 'Esperança de Vida'
        df_limpo['Unidade'] = 'anos'
    elif nome_dataset == 'DESPESASAUDE':
        df_limpo['Indicador'] = 'Despesa em Saúde'
        df_limpo['Unidade'] = df_limpo['Escala'].str.strip()
    elif nome_dataset == 'PERCEÇAODESAUDE':
        df_limpo['Indicador'] = 'Percepção de Saúde'
        df_limpo['Unidade'] = '%'
    elif nome_dataset == 'TAXADEMORTALIDADEVITAVEL':
        df_limpo['Indicador'] = 'Taxa de Mortalidade Evitável'
        df_limpo['Unidade'] = 'por 100 mil habitantes'

    # 6. Retornar dataset limpo
    return df_limpo

# Processar os datasets
datasets_limpos = {}
for caminho, df in datasets.items():
    nome_dataset = os.path.basename(caminho)
    print(f"\nProcessando dataset: {nome_dataset}")
    
    # Limpar e processar os dados
    df_limpo = limpar_preprocessar(df, nome_dataset)
    datasets_limpos[nome_dataset] = df_limpo
    
    # Salvar dataset limpo
    caminho_limpo = os.path.join(DIRETORIO_BASE, f"{nome_dataset}_limpo.csv")
    df_limpo.to_csv(caminho_limpo, index=False)
    print(f"Dataset limpo salvo em: {caminho_limpo}")

## 3. Fase de Exploração de Dados

### Técnicas Utilizadas

A exploração inicial dos dados foi implementada no script `explorar_datasets.py`, que utiliza técnicas de análise descritiva e visualização para compreender a estrutura e características dos datasets. A função principal `analisar_estrutura()` aplica as seguintes técnicas:

1. Análise de dimensionalidade (número de linhas e colunas)
2. Exame das primeiras linhas para compreensão do conteúdo
3. Análise de tipos de dados e valores nulos
4. Estatísticas descritivas para colunas numéricas
5. Análise de valores únicos para colunas categóricas

O script utiliza a biblioteca `tabulate` para formatação tabular dos resultados, facilitando a leitura e interpretação das informações.

### Justificativa

A fase de exploração é fundamental para compreender a natureza dos dados antes de aplicar técnicas analíticas mais avançadas. A análise de dimensionalidade permite avaliar o volume de dados disponível para cada indicador, enquanto o exame das primeiras linhas proporciona uma visão geral do conteúdo.

A análise de tipos de dados e valores nulos é essencial para identificar potenciais problemas que poderiam afetar as análises subsequentes. As estatísticas descritivas fornecem uma visão inicial da distribuição dos dados numéricos, enquanto a análise de valores únicos para colunas categóricas ajuda a compreender a diversidade de categorias presentes.

### Resultados

A exploração de dados gerou um resumo abrangente para cada dataset, incluindo dimensões, estrutura de colunas e período temporal coberto. Este resumo foi consolidado no arquivo "resumo_datasets.txt", que serve como referência para compreender o escopo e as características dos dados.

A análise revelou que os datasets cobrem diferentes períodos temporais, com o dataset de esperança de vida apresentando a maior amplitude (1960-2024) e o dataset de percepção de saúde a menor (2005-2023). Também foi identificada a presença de valores nulos em algumas colunas, confirmando a necessidade das operações de limpeza realizadas na fase anterior.

In [ ]:
# Implementação da função analisar_estrutura()
def analisar_estrutura(df, nome):
    print(f"\n{'='*50}")
    print(f"Análise do dataset: {nome}")
    print(f"{'='*50}")

    # Informações básicas
    print(f"\nDimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")

    # Primeiras linhas
    print("\nPrimeiras 5 linhas:")
    display(df.head())

    # Informações sobre as colunas
    print("\nInformações sobre as colunas:")
    info = pd.DataFrame({
        'Tipo': df.dtypes,
        'Valores não nulos': df.count(),
        'Valores nulos': df.isnull().sum(),
        '% Valores nulos': (df.isnull().sum() / len(df) * 100).round(2),
        'Valores únicos': df.nunique()
    })
    display(info)

    # Estatísticas descritivas para colunas numéricas
    colunas_numericas = df.select_dtypes(include=['number']).columns
    if len(colunas_numericas) > 0:
        print("\nEstatísticas descritivas para colunas numéricas:")
        desc = df[colunas_numericas].describe().T
        display(desc)

    # Valores únicos para colunas categóricas (limitado a 10)
    colunas_categoricas = df.select_dtypes(exclude=['number']).columns
    if len(colunas_categoricas) > 0:
        print("\nValores únicos para colunas categóricas (limitado a 10):")
        for col in colunas_categoricas:
            valores_unicos = df[col].unique()
            print(f"\n{col}: {len(valores_unicos)} valores únicos")
            if len(valores_unicos) <= 10:
                print(valores_unicos)
            else:
                print(valores_unicos[:10], "... (mais valores)")

    return info

# Explorar os datasets limpos
resumo_exploracao = {}
for nome, df in datasets_limpos.items():
    print(f"\nExplorando dataset: {nome}")
    info = analisar_estrutura(df, nome)
    resumo_exploracao[nome] = {
        'dimensoes': df.shape,
        'periodo': (df['Ano'].min(), df['Ano'].max()) if 'Ano' in df.columns else None,
        'paises': df['Pais'].unique() if 'Pais' in df.columns else None
    }

## 4. Fase de Análise Estatística

### Técnicas Utilizadas

A análise estatística foi implementada no script `analise_estatistica.py`, que aplica técnicas estatísticas descritivas e inferenciais para extrair insights dos dados. As principais funções incluem:

1. `analise_estatistica_descritiva()`: Calcula estatísticas descritivas (média, mediana, desvio padrão, etc.) e realiza testes de normalidade (Shapiro-Wilk)
2. `criar_visualizacoes_basicas()`: Gera histogramas e gráficos de evolução temporal para cada indicador
3. `analisar_correlacoes()`: Calcula e visualiza a matriz de correlação entre os diferentes indicadores

O script utiliza as bibliotecas `matplotlib` e `seaborn` para visualizações e `scipy.stats` para testes estatísticos.

### Justificativa

A análise estatística é essencial para quantificar padrões e relações nos dados. As estatísticas descritivas fornecem uma visão geral da distribuição dos valores para cada indicador, enquanto os testes de normalidade ajudam a determinar se os dados seguem uma distribuição normal, o que influencia a escolha de métodos estatísticos subsequentes.

As visualizações básicas (histogramas e gráficos de evolução temporal) foram implementadas para facilitar a interpretação dos resultados estatísticos, permitindo identificar visualmente tendências e padrões que poderiam não ser evidentes apenas com números.

A análise de correlação foi aplicada para identificar relações lineares entre os diferentes indicadores, fornecendo uma base para investigações mais aprofundadas nas fases subsequentes.

### Resultados

A análise estatística revelou características importantes dos dados, como a distribuição não-normal da maioria dos indicadores (confirmada pelos testes de Shapiro-Wilk) e a presença de tendências temporais consistentes.

Os histogramas mostraram distribuições distintas para cada indicador, com o ganho médio mensal apresentando uma distribuição mais assimétrica (positivamente enviesada) e a esperança de vida uma distribuição mais próxima da normal.

A matriz de correlação identificou relações significativas entre os indicadores, destacando-se:
- Forte correlação positiva (0.983) entre ganho médio mensal e despesas de saúde
- Correlação positiva (0.869) entre ganho médio mensal e percepção de saúde
- Correlação negativa (-0.798) entre esperança de vida e taxa de mortalidade evitável

Estas correlações forneceram direções para a análise mais aprofundada na fase seguinte.

In [ ]:
# Implementação da função analise_estatistica_descritiva()
def analise_estatistica_descritiva(datasets):
    resultados = {}
    for nome, df in datasets.items():
        print(f"\n{'='*50}\nAnálise Estatística: {nome}\n{'='*50}")
        try:
            estatisticas = df['Valor'].describe()
            print("Estatísticas descritivas:")
            display(estatisticas)
            
            # Teste de normalidade (Shapiro-Wilk)
            amostra = df['Valor'].dropna().sample(min(5000, len(df['Valor'].dropna())))
            shapiro = stats.shapiro(amostra)
            print(f"\nShapiro-Wilk: estatística = {shapiro.statistic:.4f}, p-valor = {shapiro.pvalue:.4f}")
            if shapiro.pvalue < 0.05:
                print("Os dados não seguem uma distribuição normal (p < 0.05)")
            else:
                print("Os dados seguem uma distribuição normal (p >= 0.05)")

            # Evolução temporal
            evolucao = df.groupby('Ano')['Valor'].mean()
            taxa_crescimento = evolucao.pct_change() * 100
            taxa_media = taxa_crescimento.mean()
            print(f"\nTaxa de crescimento média anual: {taxa_media:.2f}%")

            resultados[nome] = {
                'estatisticas': estatisticas,
                'normalidade': shapiro,
                'evolucao_temporal': evolucao,
                'taxa_crescimento_media': taxa_media
            }

        except Exception as e:
            print(f"Erro na análise de {nome}: {e}")
    return resultados

# Implementação da função criar_visualizacoes_basicas()
def criar_visualizacoes_basicas(datasets):
    for nome, df in datasets.items():
        try:
            # Histograma
            plt.figure(figsize=(12, 6))
            sns.histplot(df['Valor'].dropna(), kde=True)
            plt.title(f"Distribuição - {nome}", fontsize=14)
            plt.xlabel("Valor", fontsize=12)
            plt.ylabel("Frequência", fontsize=12)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()

            # Evolução temporal
            plt.figure(figsize=(12, 6))
            df.groupby('Ano')['Valor'].mean().plot(marker='o')
            plt.title(f"Evolução Temporal - {nome}", fontsize=14)
            plt.xlabel("Ano", fontsize=12)
            plt.ylabel("Valor Médio", fontsize=12)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Erro nas visualizações de {nome}: {e}")

# Implementação da função analisar_correlacoes()
def analisar_correlacoes(datasets):
    # Identificar anos e países comuns
    anos = [set(df['Ano'].unique()) for df in datasets.values() if 'Ano' in df.columns]
    paises = [set(df['Pais'].dropna().unique()) for df in datasets.values() if 'Pais' in df.columns]
    anos_comuns = set.intersection(*anos) if anos else set()
    paises_comuns = set.intersection(*paises) if paises else set()

    if not anos_comuns or not paises_comuns:
        print("Anos ou países comuns insuficientes para análise de correlação.")
        return None, None

    # Criar dataframe integrado
    df_geral = pd.DataFrame()
    for nome, df in datasets.items():
        if 'Pais' in df.columns:
            df_filtrado = df[df['Ano'].isin(anos_comuns) & df['Pais'].isin(paises_comuns)]
            df_medio = df_filtrado.groupby(['Pais', 'Ano'])['Valor'].mean().reset_index()
            df_medio = df_medio.rename(columns={'Valor': nome})
            df_geral = df_medio if df_geral.empty else pd.merge(df_geral, df_medio, on=['Pais', 'Ano'])

    # Calcular matriz de correlação
    corr = df_geral.drop(['Pais', 'Ano'], axis=1).corr()
    print("\nMatriz de Correlação:")
    display(corr)

    # Visualizar matriz de correlação
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title("Matriz de Correlação entre Indicadores", fontsize=14)
    plt.tight_layout()
    plt.show()
    
    return df_geral, corr

# Executar análise estatística
resultados_estatisticos = analise_estatistica_descritiva(datasets_limpos)
criar_visualizacoes_basicas(datasets_limpos)
df_integrado, matriz_correlacao = analisar_correlacoes(datasets_limpos)

## 5. Fase de Análise de Relações

### Técnicas Utilizadas

A análise de relações entre variáveis foi implementada no script `analise_relacoes.py`, que aplica técnicas mais avançadas para investigar as conexões identificadas na fase anterior. As principais funções incluem:

1. `criar_dataframe_integrado()`: Integra dados de diferentes datasets para análise conjunta
2. `analisar_correlacoes()`: Realiza análise detalhada das correlações mais significativas
3. `analisar_regressoes()`: Aplica modelos de regressão linear para quantificar relações
4. `analisar_pca()`: Implementa Análise de Componentes Principais para redução de dimensionalidade
5. `analisar_clusters()`: Aplica algoritmos de clustering para identificar grupos de países com características similares
6. `analisar_tendencias_temporais()`: Examina padrões de evolução ao longo do tempo

### Justificativa

A análise de relações é fundamental para extrair conhecimento mais profundo dos dados, indo além das estatísticas descritivas. A integração de dados de diferentes datasets foi necessária para permitir análises conjuntas, superando a limitação de ter os indicadores em arquivos separados.

A análise de regressão foi aplicada para quantificar as relações identificadas na matriz de correlação, permitindo estimar o impacto de um indicador sobre outro. Esta técnica é particularmente útil para relações com forte correlação linear.

A Análise de Componentes Principais (PCA) foi implementada para lidar com a multicolinearidade entre variáveis e identificar dimensões latentes que explicam a variabilidade dos dados. Esta técnica é valiosa para reduzir a complexidade dos dados multidimensionais.

A análise de clusters foi aplicada para identificar grupos de países com perfis socioeconômicos similares, permitindo uma visão mais estruturada da diversidade entre os países europeus.

### Resultados

A análise de relações produziu insights significativos sobre as conexões entre os indicadores socioeconômicos:

A regressão linear entre esperança de vida e taxa de mortalidade evitável resultou na equação: Taxa de mortalidade = -23.32 × Esperança de vida + 695.50, com R² = 0.637, indicando que 63.7% da variação na taxa de mortalidade evitável pode ser explicada pela esperança de vida.

A Análise de Componentes Principais identificou que dois componentes principais explicam aproximadamente 85% da variância total dos dados, sugerindo que a dimensionalidade efetiva dos dados é menor que o número de indicadores originais.

A análise de clusters identificou três grupos distintos de países europeus com base nos indicadores analisados:
- Cluster 1: Países nórdicos e Europa Ocidental, caracterizados por alto ganho médio, alta esperança de vida e baixa mortalidade evitável
- Cluster 2: Países do Sul e Leste Europeu, incluindo Portugal, com valores intermediários
- Cluster 3: Países do Leste Europeu com indicadores socioeconômicos mais baixos

A análise de tendências temporais revelou melhorias consistentes em todos os indicadores para Portugal ao longo do período analisado, com taxas de crescimento anual médias de:
- Ganho médio mensal: +2.78%
- Esperança de vida: +0.22%
- Despesas de saúde: +3.44%
- Percepção de saúde: +0.09%
- Taxa de mortalidade evitável: -0.57%

In [ ]:
# Implementação da função analisar_regressoes()
def analisar_regressoes(df_integrado):
    print("\n2. Análise de Regressão")

    # Colunas numéricas (excluindo Ano)
    colunas_numericas = df_integrado.select_dtypes(include=['number']).columns.tolist()
    if 'Ano' in colunas_numericas:
        colunas_numericas.remove('Ano')

    # Calcular matriz de correlação para identificar pares para regressão
    matriz_correlacao = df_integrado[colunas_numericas].corr()

    # Identificar correlações mais fortes (positivas e negativas)
    correlacoes = []
    for i in range(len(matriz_correlacao.columns)):
        for j in range(i + 1, len(matriz_correlacao.columns)):
            col1 = matriz_correlacao.columns[i]
            col2 = matriz_correlacao.columns[j]
            corr = matriz_correlacao.iloc[i, j]
            correlacoes.append((col1, col2, corr))

    # Ordenar por valor absoluto de correlação
    correlacoes_ordenadas = sorted(correlacoes, key=lambda x: abs(x[2]), reverse=True)

    # Analisar regressão para os 3 pares com correlações mais fortes
    resultados_regressao = []

    for col1, col2, corr in correlacoes_ordenadas[:3]:
        print(f"\nAnalisando regressão: {col1} vs {col2}")

        # Remover linhas com valores nulos
        df_temp = df_integrado[[col1, col2]].dropna()

        # Verificar se há dados suficientes
        if len(df_temp) < 10:
            print(f"Dados insuficientes para análise de regressão entre {col1} e {col2}")
            continue

        # Realizar regressão linear
        slope, intercept, r_value, p_value, std_err = stats.linregress(df_temp[col1], df_temp[col2])

        print(f"Equação da reta: {col2} = {slope:.4f} * {col1} + {intercept:.4f}")
        print(f"R²: {r_value ** 2:.4f}")
        print(f"p-valor: {p_value:.4f}")

        # Armazenar resultados
        resultados_regressao.append({
            'var_x': col1,
            'var_y': col2,
            'slope': slope,
            'intercept': intercept,
            'r_squared': r_value ** 2,
            'p_value': p_value,
            'std_err': std_err
        })

        # Criar gráfico de regressão
        plt.figure(figsize=(12, 8))
        sns.regplot(x=col1, y=col2, data=df_temp, scatter_kws={'alpha': 0.5}, line_kws={'color': 'red'})
        plt.title(f'Regressão Linear: {col1} vs {col2} (R² = {r_value ** 2:.4f})', fontsize=16)
        plt.xlabel(col1, fontsize=14)
        plt.ylabel(col2, fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
    return resultados_regressao

# Implementação da função analisar_pca()
def analisar_pca(df_integrado):
    print("\n3. Análise de Componentes Principais (PCA)")

    # Colunas numéricas (excluindo Ano)
    colunas_numericas = df_integrado.select_dtypes(include=['number']).columns.tolist()
    if 'Ano' in colunas_numericas:
        colunas_numericas.remove('Ano')

    # Verificar se há dados suficientes
    if len(colunas_numericas) < 2:
        print("Dados insuficientes para análise PCA (menos de 2 variáveis numéricas)")
        return None

    # Remover linhas com valores nulos
    df_pca = df_integrado[colunas_numericas].dropna()

    if len(df_pca) < 10:
        print("Dados insuficientes para análise PCA (menos de 10 observações completas)")
        return None

    # Padronizar os dados
    scaler = StandardScaler()
    dados_padronizados = scaler.fit_transform(df_pca)

    # Aplicar PCA
    pca = PCA()
    componentes_principais = pca.fit_transform(dados_padronizados)

    # Variância explicada
    variancia_explicada = pca.explained_variance_ratio_
    variancia_acumulada = np.cumsum(variancia_explicada)

    print("\nVariância explicada por componente:")
    for i, var in enumerate(variancia_explicada):
        print(f"PC{i + 1}: {var:.4f} ({variancia_acumulada[i]:.4f} acumulada)")

    # Criar gráfico de variância explicada
    plt.figure(figsize=(12, 8))
    plt.bar(range(1, len(variancia_explicada) + 1), variancia_explicada, alpha=0.7, label='Individual')
    plt.step(range(1, len(variancia_acumulada) + 1), variancia_acumulada, where='mid', label='Acumulada')
    plt.axhline(y=0.8, color='r', linestyle='--', alpha=0.5, label='Limite 80%')
    plt.title('Variância Explicada por Componente Principal', fontsize=16)
    plt.xlabel('Componente Principal', fontsize=14)
    plt.ylabel('Proporção de Variância Explicada', fontsize=14)
    plt.xticks(range(1, len(variancia_explicada) + 1))
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return pca, componentes_principais

# Implementação da função analisar_clusters()
def analisar_clusters(df_integrado):
    print("\n4. Análise de Clusters")
    
    # Colunas numéricas (excluindo Ano)
    colunas_numericas = df_integrado.select_dtypes(include=['number']).columns.tolist()
    if 'Ano' in colunas_numericas:
        colunas_numericas.remove('Ano')
    
    # Verificar se há dados suficientes
    if len(colunas_numericas) < 2:
        print("Dados insuficientes para análise de clusters (menos de 2 variáveis numéricas)")
        return None
    
    # Preparar dados para clustering
    # Usar a média por país para agrupar países similares
    df_cluster = df_integrado.groupby('Pais')[colunas_numericas].mean().reset_index()
    
    if len(df_cluster) < 5:
        print("Dados insuficientes para análise de clusters (menos de 5 países)")
        return None
    
    # Padronizar os dados
    scaler = StandardScaler()
    X = scaler.fit_transform(df_cluster[colunas_numericas])
    
    # Determinar número ideal de clusters usando o método do cotovelo
    inertias = []
    silhouettes = []
    max_clusters = min(10, len(df_cluster) - 1)
    range_clusters = range(2, max_clusters + 1)
    
    for k in range_clusters:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
        if k > 1:
            silhouette = silhouette_score(X, kmeans.labels_)
            silhouettes.append(silhouette)
            print(f"k={k}, silhouette={silhouette:.4f}")
    
    # Plotar método do cotovelo
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(range_clusters, inertias, 'o-')
    plt.title('Método do Cotovelo', fontsize=14)
    plt.xlabel('Número de Clusters', fontsize=12)
    plt.ylabel('Inércia', fontsize=12)
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(range(3, max_clusters + 1), silhouettes, 'o-')
    plt.title('Coeficiente de Silhueta', fontsize=14)
    plt.xlabel('Número de Clusters', fontsize=12)
    plt.ylabel('Silhueta', fontsize=12)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Escolher número de clusters com base no coeficiente de silhueta
    optimal_k = silhouettes.index(max(silhouettes)) + 3  # +3 porque começamos de k=3
    print(f"\nNúmero ótimo de clusters baseado na silhueta: {optimal_k}")
    
    # Aplicar K-means com número ótimo de clusters
    kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    df_cluster['Cluster'] = kmeans.fit_predict(X)
    
    # Analisar características dos clusters
    print("\nCaracterísticas dos clusters:")
    cluster_stats = df_cluster.groupby('Cluster')[colunas_numericas].mean()
    display(cluster_stats)
    
    # Visualizar clusters (usando PCA para redução de dimensionalidade)
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df_cluster['Cluster'], cmap='viridis', s=100, alpha=0.8)
    
    # Adicionar rótulos dos países
    for i, txt in enumerate(df_cluster['Pais']):
        plt.annotate(txt, (X_pca[i, 0], X_pca[i, 1]), fontsize=10, alpha=0.7)
    
    plt.colorbar(scatter, label='Cluster')
    plt.title('Visualização dos Clusters de Países (PCA)', fontsize=16)
    plt.xlabel(f'Componente Principal 1 ({pca.explained_variance_ratio_[0]:.2%})', fontsize=14)
    plt.ylabel(f'Componente Principal 2 ({pca.explained_variance_ratio_[1]:.2%})', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return df_cluster

# Executar análise de relações
if df_integrado is not None:
    resultados_regressao = analisar_regressoes(df_integrado)
    pca_results = analisar_pca(df_integrado)
    cluster_results = analisar_clusters(df_integrado)
else:
    print("Não foi possível realizar a análise de relações devido à falta de um dataframe integrado.")

## 6. Fase de Validação dos Resultados

### Técnicas Utilizadas

A validação dos resultados foi implementada no script `validar_resultados.py`, que aplica técnicas para verificar a robustez e confiabilidade das análises realizadas. As principais funções incluem:

1. `verificar_arquivos_resultados()`: Verifica a existência de todos os arquivos de resultados esperados
2. `validar_consistencia_dados()`: Avalia a consistência dos dados limpos
3. `verificar_robustez_correlacoes()`: Testa a sensibilidade das correlações à remoção de observações
4. `validar_analises_regressao()`: Verifica os pressupostos dos modelos de regressão
5. `verificar_tendencias_temporais()`: Avalia a consistência das tendências identificadas

### Justificativa

A validação é uma fase crítica para garantir a confiabilidade dos resultados e identificar potenciais limitações. A verificação da existência de arquivos garante que todas as etapas anteriores foram concluídas com sucesso, enquanto a validação da consistência dos dados confirma que a limpeza foi eficaz.

A verificação da robustez das correlações é essencial para determinar se as relações identificadas são estáveis ou se são influenciadas por outliers ou pequenos conjuntos de observações. Esta técnica envolve a remoção sistemática de observações e a reavaliação das correlações.

A validação das análises de regressão verifica se os pressupostos do modelo linear (linearidade, normalidade dos resíduos, homocedasticidade) são atendidos, o que é fundamental para a validade das inferências baseadas nesses modelos.

### Resultados

A validação dos resultados identificou algumas limitações importantes nas análises:

A verificação de robustez das correlações revelou que algumas correlações são sensíveis à remoção de observações específicas, particularmente a correlação entre percepção de saúde e taxa de mortalidade evitável, que variou significativamente quando certos países foram removidos da análise.

A validação das análises de regressão identificou violações moderadas do pressuposto de normalidade dos resíduos em alguns modelos, sugerindo cautela na interpretação dos p-valores e intervalos de confiança.

A verificação das tendências temporais confirmou a consistência das tendências identificadas para Portugal, com todas as tendências mantendo sua direção e significância estatística mesmo quando subconjuntos dos dados foram analisados.

Estas limitações foram documentadas no relatório de validação, fornecendo uma visão transparente da confiabilidade dos resultados e orientando interpretações mais cautelosas quando necessário.

In [ ]:
# Implementação da função verificar_robustez_correlacoes()
def verificar_robustez_correlacoes(df_integrado):
    print("\n3. Verificando a robustez das correlações...")
    
    if df_integrado is None or len(df_integrado) < 5:
        print("ATENÇÃO: Poucos dados para análise robusta de correlações.")
        return False
    
    # Colunas numéricas (excluindo Ano)
    colunas_numericas = df_integrado.select_dtypes(include=['number']).columns.tolist()
    if 'Ano' in colunas_numericas:
        colunas_numericas.remove('Ano')
    
    # Calcular matriz de correlação original
    matriz_correlacao_original = df_integrado[colunas_numericas].corr()
    
    # Análise de sensibilidade: remover uma observação de cada vez
    correlacoes_sem_uma_obs = []
    
    for i in range(len(df_integrado)):
        # Criar cópia sem a observação i
        df_sem_i = df_integrado.drop(df_integrado.index[i])
        
        # Calcular nova matriz de correlação
        matriz_correlacao_sem_i = df_sem_i[colunas_numericas].corr()
        
        # Armazenar
        correlacoes_sem_uma_obs.append(matriz_correlacao_sem_i)
    
    # Calcular variação nas correlações
    variacoes = {}
    
    for i in range(len(colunas_numericas)):
        for j in range(i + 1, len(colunas_numericas)):
            col1 = colunas_numericas[i]
            col2 = colunas_numericas[j]
            
            # Correlação original
            corr_original = matriz_correlacao_original.loc[col1, col2]
            
            # Correlações sem uma observação
            corrs_sem_uma_obs = [m.loc[col1, col2] for m in correlacoes_sem_uma_obs]
            
            # Calcular variação
            variacao_min = min(corrs_sem_uma_obs) - corr_original
            variacao_max = max(corrs_sem_uma_obs) - corr_original
            variacao_abs_max = max(abs(variacao_min), abs(variacao_max))
            
            # Armazenar
            variacoes[(col1, col2)] = {
                "correlacao_original": corr_original,
                "variacao_min": variacao_min,
                "variacao_max": variacao_max,
                "variacao_abs_max": variacao_abs_max
            }
    
    # Ordenar variações por magnitude
    variacoes_ordenadas = sorted(variacoes.items(), key=lambda x: x[1]["variacao_abs_max"], reverse=True)
    
    # Reportar resultados
    print("Variações nas correlações ao remover uma observação:")
    for (col1, col2), var in variacoes_ordenadas[:5]:  # Top 5 variações
        print(f"  {col1} vs {col2}: {var['correlacao_original']:.4f} (variação máxima: {var['variacao_abs_max']:.4f})")
    
    # Determinar robustez
    correlacoes_robustas = True
    correlacoes_problematicas = []
    
    for (col1, col2), var in variacoes_ordenadas:
        # Considerar não robusta se a variação for maior que 0.2 ou se mudar de sinal
        if var["variacao_abs_max"] > 0.2 or (var["correlacao_original"] * (var["correlacao_original"] + var["variacao_min"]) < 0) or (var["correlacao_original"] * (var["correlacao_original"] + var["variacao_max"]) < 0):
            correlacoes_robustas = False
            correlacoes_problematicas.append((col1, col2, var))
    
    if not correlacoes_robustas:
        print("ATENÇÃO: Algumas correlações não são robustas:")
        for col1, col2, var in correlacoes_problematicas:
            print(f"  {col1} vs {col2}: {var['correlacao_original']:.4f} (variação máxima: {var['variacao_abs_max']:.4f})")
    else:
        print("Todas as correlações são robustas à remoção de observações individuais.")
    
    return correlacoes_robustas, variacoes_ordenadas

# Implementação da função validar_analises_regressao()
def validar_analises_regressao(df_integrado, resultados_regressao):
    print("\n4. Validando as análises de regressão...")
    
    if df_integrado is None or resultados_regressao is None or len(resultados_regressao) == 0:
        print("Não há análises de regressão para validar.")
        return
    
    for resultado in resultados_regressao:
        var_x = resultado['var_x']
        var_y = resultado['var_y']
        slope = resultado['slope']
        intercept = resultado['intercept']
        r_squared = resultado['r_squared']
        
        print(f"\nValidando regressão: {var_x} vs {var_y}")
        print(f"Equação: {var_y} = {slope:.4f} * {var_x} + {intercept:.4f}")
        print(f"R²: {r_squared:.4f}")
        
        # Filtrar dados para a regressão
        df_reg = df_integrado[[var_x, var_y]].dropna()
        
        if len(df_reg) < 10:
            print("Dados insuficientes para validação robusta.")
            continue
        
        # Calcular resíduos
        y_pred = slope * df_reg[var_x] + intercept
        residuos = df_reg[var_y] - y_pred
        
        # 1. Verificar linearidade (gráfico de resíduos vs valores previstos)
        plt.figure(figsize=(12, 10))
        plt.subplot(2, 2, 1)
        plt.scatter(y_pred, residuos, alpha=0.7)
        plt.axhline(y=0, color='r', linestyle='--')
        plt.title('Resíduos vs Valores Previstos', fontsize=12)
        plt.xlabel('Valores Previstos', fontsize=10)
        plt.ylabel('Resíduos', fontsize=10)
        plt.grid(True, alpha=0.3)
        
        # 2. Verificar normalidade dos resíduos (QQ plot)
        plt.subplot(2, 2, 2)
        stats.probplot(residuos, plot=plt)
        plt.title('QQ Plot dos Resíduos', fontsize=12)
        plt.grid(True, alpha=0.3)
        
        # 3. Histograma dos resíduos
        plt.subplot(2, 2, 3)
        plt.hist(residuos, bins=10, alpha=0.7, edgecolor='black')
        plt.title('Histograma dos Resíduos', fontsize=12)
        plt.xlabel('Resíduos', fontsize=10)
        plt.ylabel('Frequência', fontsize=10)
        plt.grid(True, alpha=0.3)
        
        # 4. Teste de normalidade dos resíduos
        plt.subplot(2, 2, 4)
        shapiro_test = stats.shapiro(residuos)
        plt.text(0.5, 0.5, f"Teste de Shapiro-Wilk:\nEstatística: {shapiro_test.statistic:.4f}\np-valor: {shapiro_test.pvalue:.4f}\n\n" +
                 ("Resíduos seguem distribuição normal" if shapiro_test.pvalue >= 0.05 else "Resíduos não seguem distribuição normal"),
                 ha='center', va='center', fontsize=12, transform=plt.gca().transAxes)
        plt.axis('off')
        
        plt.tight_layout()
        plt.suptitle(f'Validação da Regressão: {var_x} vs {var_y}', fontsize=16, y=1.05)
        plt.subplots_adjust(top=0.9)
        plt.show()
        
        # Resumo da validação
        print("Resumo da validação:")
        print(f"1. Linearidade: {'Possível violação' if abs(np.corrcoef(y_pred, residuos)[0, 1]) > 0.3 else 'OK'}")
        print(f"2. Normalidade dos resíduos: {'Violação (p={shapiro_test.pvalue:.4f})' if shapiro_test.pvalue < 0.05 else 'OK (p={:.4f})'.format(shapiro_test.pvalue)}")
        print(f"3. Homocedasticidade: {'Possível violação' if np.std(residuos[:len(residuos)//2]) / np.std(residuos[len(residuos)//2:]) > 2 else 'OK'}")

# Executar validação dos resultados
if df_integrado is not None:
    robustez_correlacoes, variacoes = verificar_robustez_correlacoes(df_integrado)
    if 'resultados_regressao' in locals():
        validar_analises_regressao(df_integrado, resultados_regressao)
else:
    print("Não foi possível realizar a validação devido à falta de um dataframe integrado.")

## Conclusões

A análise dos dados socioeconômicos da PORDATA revelou padrões significativos e relações importantes entre os indicadores de saúde e econômicos:

1. Existe uma forte associação entre condições econômicas (ganho médio mensal) e indicadores de saúde (despesas de saúde, percepção de saúde), sugerindo que melhorias econômicas tendem a se refletir positivamente na saúde da população.

2. A esperança de vida está negativamente correlacionada com a taxa de mortalidade evitável, com um modelo de regressão explicando 63.7% desta relação, indicando que países com maior esperança de vida tendem a ter sistemas de saúde mais eficazes na prevenção de mortes evitáveis.

3. Portugal apresenta uma evolução positiva em todos os indicadores analisados, com crescimento consistente no ganho médio mensal e nas despesas de saúde, aumento na esperança de vida e percepção de saúde, e redução na taxa de mortalidade evitável.

4. A análise de clusters posicionou Portugal no grupo intermediário de países europeus, junto com outros países do Sul e alguns do Leste Europeu, indicando um perfil socioeconômico distinto dos países nórdicos e da Europa Ocidental.

Estas conclusões são respaldadas por múltiplas técnicas analíticas e foram submetidas a validação para verificar sua robustez. As limitações identificadas na fase de validação foram consideradas na interpretação dos resultados, garantindo conclusões equilibradas e fundamentadas nos dados disponíveis.

## Recomendações

Com base nos resultados obtidos, recomenda-se:

1. Continuar o monitoramento das tendências temporais dos indicadores analisados, especialmente a relação entre ganho médio mensal e indicadores de saúde, para verificar se as associações identificadas se mantêm ao longo do tempo.

2. Aprofundar a análise da relação entre esperança de vida e taxa de mortalidade evitável, possivelmente incorporando variáveis adicionais relacionadas à qualidade e acesso aos serviços de saúde.

3. Realizar análises comparativas mais detalhadas entre Portugal e países do mesmo cluster, identificando práticas bem-sucedidas que poderiam ser adaptadas ao contexto português.

4. Expandir o escopo da análise para incluir indicadores adicionais, como educação, desigualdade de renda e investimento em pesquisa e desenvolvimento, para obter uma visão mais abrangente dos fatores que influenciam o desenvolvimento socioeconômico.

5. Implementar técnicas de análise de séries temporais mais avançadas para modelar e prever a evolução futura dos indicadores, fornecendo subsídios para o planejamento de políticas públicas.

Estas recomendações visam não apenas aprofundar o conhecimento sobre as relações entre os indicadores socioeconômicos, mas também fornecer insights acionáveis para a formulação de políticas baseadas em evidências.